<h1><center><b>Tecnologias de Processamento de Dados - 2019/2020<b></center></h1>
<h2><center>Fase 3</center></h2> 
<h2><center>Grupo 21</center><h2>
<table>
  <tr>
    <th><h3><b>Nome</b></h3></th>
    <th><h3><b>Número</b></h3></th>
    <th><h3><b>Horas</b></h3></th>
  </tr>
  <tr>
    <td><h3>Diogo Silva</h3></td>
    <td><h3>54131</h3></td>
    <td><h3>14</h3></td>
  </tr>
  <tr>
    <td><h3>Nuno Silva</h3></td>
    <td><h3>55109</h3></td>
    <td><h3>14</h3></td>
  </tr>
  <tr>
    <td><h3>Pedro Russo</h3></td>
    <td><h3>55101</h3></td>
    <td><h3>14</h3></td>
  </tr>
</table>

<h2> 1. OLAP queries </h2>

In [0]:
import pandas as pd
import psycopg2

* <h3>Qual o número de acidentes registados por mês, por localização e por modelo de aeronave?</h3>
Dimensões: DIMAircraft, DIMLocation, DIMDate
<br>
Atributos: CalendarMonth, Model, Location
<br>
Medidas: FKAirCraft, FKDate, FKLocation e contagem de factos

In [0]:
connection = None
command = ("""SELECT CalendarMonth,Model,Location,COUNT(*) FROM DIMAircraft,Fact,DIMLocation,DIMDate 
                      WHERE FKAirCraft = AircraftID AND DateID = FKDate AND LocationID = FKLocation 
                      GROUP BY CalendarMonth, Location, Model 
                      ORDER BY COUNT(*) DESC""")

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()

  table = []
 
  for ed in dataDB: 
    obj = {
        'CalendarMonth': ed[0],
        'Model': ed[1],
        'Location': ed[2],
        'Count': ed[3]
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,CalendarMonth,Model,Location,Count
0,August,172,"Daytona Beach, FL",2
1,December,208,"Victoria, TX",1
2,September,210,"Marathon, FL",1
3,October,PA32R,"Paducah, KY",1
4,August,MURPHY MOOSE,"Marysville, WA",1
...,...,...,...,...
495,October,AR-1,"Brooksville, FL",1
496,November,M20M,"College Park, MD",1
497,December,PA 31T,"LAFAYETTE, LA",1
498,August,G103,"San Antonio, TX",1


* <h3>Qual é o número de acidentes por país e por fase de voo?</h3>
  Dimensões: DIMLocation, DIMAccident
<br>
  Atributos: Country, BroadPhaseOfFLight
<br>
  Medidas: FKLocation, FKAccident e contagem de factos

In [0]:
connection = None
command = ("""SELECT Country,BroadPhaseOfFlight,COUNT(*) FROM DIMLocation,Fact,DIMAccident 
                      WHERE FKAccident = AccidentID AND LocationID = FKLocation 
                      GROUP BY Country,BroadPhaseOfFlight 
                      ORDER BY COUNT(*) DESC""")

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()

  table = []
 
  for ed in dataDB: 
    obj = {
        'Country': ed[0],
        'BroadPhaseOfFlight': ed[1],
        'Amount': ed[2]
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,Country,BroadPhaseOfFlight,Amount
0,United States of America,Unknown,117
1,United States of America,TAKEOFF,92
2,United States of America,LANDING,84
3,United States of America,MANEUVERING,41
4,United States of America,CRUISE,40
...,...,...,...
58,Egypt,MANEUVERING,1
59,India,APPROACH,1
60,Saudi Arabia,TAKEOFF,1
61,Chile,STANDING,1


* <h3> Qual é o número de mortes por semestre, por dano na aeronave e por fase de voo? </h3>
   Dimensões: DIMDate, DIMAccident
   <br>
   Atributos: Semester, AircraftDamage, BroadPhaseOfFlight
   <br>
   Medidas: FKDate, FKAccident, TotalFatalInjuries

In [0]:
connection = None
command = ("""SELECT Semester,AircraftDamage,BroadPhaseOfFlight,SUM(TotalFatalInjuries) FROM Fact,DIMAccident,DIMDate 
                      WHERE AccidentID = FKAccident AND DateID = FKDate AND TotalFatalInjuries > -1
                      GROUP BY Semester,AircraftDamage,BroadPhaseOfFlight
                      ORDER BY SUM(TotalFatalInjuries) DESC""")

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  
  table = []
 
  for ed in dataDB: 
    obj = {
        'Semester': ed[0],
        'AircraftDamage': ed[1],
        'BroadPhaseOfFlight': ed[2],
        'TotalDeaths': ed[3]
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,Semester,AircraftDamage,BroadPhaseOfFlight,TotalDeaths
0,2,Destroyed,TAKEOFF,45
1,2,Destroyed,APPROACH,25
2,2,Destroyed,UNKNOWN,21
3,2,Substantial,TAKEOFF,20
4,2,Destroyed,Unknown,19
5,2,Destroyed,CRUISE,19
6,2,Substantial,APPROACH,18
7,2,Substantial,MANEUVERING,16
8,2,Substantial,CRUISE,13
9,2,Destroyed,MANEUVERING,13


* <h3> Que modelos de aeronave tiveram mais acidentes por ano e por semestre? </h3>
   Dimensões: DIMDate, DIMAircraft
   <br>
   Atributos: Model, CalendarYear, Semester
   <br>
   Medidas: FKDate, FKAircraft e contagem de factos

In [0]:
connection = None
command = ("""SELECT Model,CalendarYear,Semester,COUNT(*) FROM Fact,DIMDate,DIMAircraft 
                    WHERE AircraftID = FKAirCraft AND FKDate = DateID 
                    GROUP BY Model,CalendarYear,Semester 
                    ORDER BY COUNT(*) DESC """)

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  
  table = []
 
  for ed in dataDB: 
    obj = {
        'Model': ed[0],
        'CalendarYear': ed[1],
        'Semester': ed[2],
        'Amount': ed[3]
        
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,Model,CalendarYear,Semester,Amount
0,172,2019,2,34
1,PA28,2019,2,17
2,PA18,2019,2,10
3,R44,2019,2,10
4,SR22,2019,2,10
...,...,...,...,...
285,TU206,2019,2,1
286,RV 10,2019,2,1
287,Midget 0100,2019,2,1
288,AT802,2019,2,1


* <h3> Qual é o número de acidentes por número de motores e por tipo de motor? </h3>
   Dimensões: DIMAircraft
   <br>
   Atributos: NumberOfEngines, EngineType
   <br>
   Medidas: FKAircraft e contagem de factos

In [0]:
connection = None
command = ("""SELECT NumberOfEngines,EngineType,COUNT(*) FROM Fact,DIMAircraft 
                      WHERE AircraftID = FKAirCraft  
                      GROUP BY NumberOfEngines,EngineType 
                      ORDER BY COUNT(*) DESC """)

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  
  table = []
 
  for ed in dataDB: 
    obj = {
        'NumberOfEngines': ed[0],
        'EngineType': ed[1],
        'Amount': ed[2]
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,NumberOfEngines,EngineType,Amount
0,1,Reciprocating,229
1,-1,Unknown,120
2,1,Unknown,57
3,2,Reciprocating,20
4,-1,Reciprocating,18
5,1,Turbo Prop,14
6,1,Turbo Shaft,9
7,2,Unknown,9
8,2,Turbo Fan,5
9,-1,Turbo Fan,4


* <h3> O número de acidentes aumenta de acordo com o trimeste do ano? </h3>
   Dimensões: DimDate
   <br>
   Atributos: Trimester
   <br>
   Medidas: FKDate e contagem de factos

In [0]:
connection = None
command = ("""Select Trimester,COUNT(*) From DimDate,Fact
                    WHERE FKDate = DateID
                    GROUP BY Trimester 
                """)

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  
  table = []
 
  for ed in dataDB: 
    obj = {
        'Trimestre': ed[0],
        'Amount': ed[1]
        
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,Trimestre,Amount
0,4,246
1,3,255


* <h3> Existe uma condição (visual ou instrumental), por país e por mês, que torne os acidentes mais favoráveis? </h3>
   Dimensões: DimDate, DimAccident, DimLocation
   <br>
   Atributos: AccidentId, WeatherCondition, Country, CalendarMonth
   <br>
   Medidas: FKDate, FKAccident, FKLocation e contagem de factos

In [0]:
connection = None
command = ("""SELECT WeatherCondition, Country, CalendarMonth, COUNT(*) FROM Fact,DIMAccident,DIMDate,DIMLocation
                      WHERE AccidentID = FKAccident and LocationID = FKLocation and DateID = FKDate
                      GROUP BY WeatherCondition, Country, CalendarMonth 
                      ORDER BY COUNT(*) DESC """)

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  table = []
 
  for ed in dataDB: 
    obj = {
        'WeatherCondition': ed[0],
        'Country': ed[1],
        'CalendarMonth': ed[2],
        'Amount': ed[3]
        
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,WeatherCondition,Country,CalendarMonth,Amount
0,VMC,United States of America,September,107
1,VMC,United States of America,October,92
2,VMC,United States of America,August,72
3,VMC,United States of America,November,53
4,VMC,United States of America,December,22
...,...,...,...,...
68,Unknown,Croatia,September,1
69,Unknown,Saudi Arabia,September,1
70,Unknown,United Kingdom,October,1
71,UNK,Brazil,August,1


* <h3> Os acidentes aumentam de acordo com a categoria de aeronave? </h3>
   Dimensões: DimAccident,  DimAircraft
   <br>
   Atributos: AccidentId, AircraftCategory
   <br>
   Medidas: FKAccident, FKAircraft e contagem de factos


In [0]:
connection = None
command = ("""SELECT AircraftCategory, COUNT(*) FROM Fact,DIMAircraft,DIMAccident 
                     WHERE AccidentID = FKAccident AND AircraftID = FKAirCraft  
                     GROUP BY AircraftCategory 
                     ORDER BY COUNT(*) DESC""")

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
  table = []
 
  for ed in dataDB: 
    obj = {
        'AircraftCategory': ed[0],
        'Amount': ed[1],
        
    }
    table.append(obj)
  df = pd.DataFrame(table)
  display(df)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

,AircraftCategory,Amount
0,Airplane,413
1,Helicopter,48
2,Unknown,15
3,Gyroplane,7
4,Glider,7
5,Balloon,5
6,Powered Parachute,2
7,Weight-Shift,2
8,Ultralight,2


<h2> 2. Bottlenecks </h2>

Foram executadas análises de performance, utilizando para tal, o explain analyze, com o objetivo de perceber se existia algum bottleneck. Os resultados destas análises demonstram que não existem bottlenecks e necessidade de, por exemplo, o uso de índices, devido ao reduzido número de dados inseridos na base de dados por conta do bottleneck da fase anterior (o uso do geopy na dimensão Location).

Exemplo de uma análise de performance:

In [0]:

connection = None
command = ("""EXPLAIN ANALYZE SELECT CalendarMonth,Model,Location,COUNT(*) FROM DIMAircraft,Fact,DIMLocation,DIMDate 
                      WHERE FKAirCraft = AircraftID AND DateID = FKDate AND LocationID = FKLocation 
                      GROUP BY CalendarMonth, Location, Model ORDER BY COUNT(*) DESC""")

try:
  connection = psycopg2.connect(user = "kpswytoq",
                              password = "wa-gFIB1OuYVbDBnsbahGo02W9v2xrOs",
                              host = "kandula.db.elephantsql.com",
                              port = "5432",
                              database = "kpswytoq")
  cur = connection.cursor()
  
  cur.execute(command)
  dataDB = cur.fetchall()
  colnames = [desc[0] for desc in cur.description]
  cur.close()
 
  for ed in dataDB: 
    print(ed)
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
finally:
  if connection is not None:
    connection.close()

('Sort  (cost=104.73..105.98 rows=501 width=35) (actual time=1.566..1.596 rows=500 loops=1)',)
('  Sort Key: (count(*)) DESC',)
('  Sort Method: quicksort  Memory: 65kB',)
('  ->  HashAggregate  (cost=77.25..82.26 rows=501 width=35) (actual time=1.333..1.426 rows=500 loops=1)',)
('        Group Key: dimdate.calendarmonth, dimlocation.location, dimaircraft.model',)
('        ->  Hash Join  (cost=51.22..72.24 rows=501 width=27) (actual time=0.581..1.126 rows=501 loops=1)',)
('              Hash Cond: ((fact.fkdate)::text = (dimdate.dateid)::text)',)
('              ->  Hash Join  (cost=45.21..64.87 rows=501 width=52) (actual time=0.502..0.905 rows=501 loops=1)',)
('                    Hash Cond: ((fact.fklocation)::text = (dimlocation.locationid)::text)',)
('                    ->  Hash Join  (cost=19.09..37.43 rows=501 width=71) (actual time=0.232..0.495 rows=501 loops=1)',)
('                          Hash Cond: ((fact.fkaircraft)::text = (dimaircraft.aircraftid)::text)',)
('          

Dentro de 1 ano pensamos que a taxa de crescimento de dados será menor, em comparação com o que se verificou em anos anteriores, devido ao COVID-19, que pode continuar a limitar a quantidade de voos a serem efetuados, aumento de trabalhos remotos, entre outras mudanças. Nos próximos 5 a 10 anos, a expectativa é que o data warehouse continue a crescer, embora esperemos que com o avanço da tecnologia e o aumento da quantidade de informação disponível em torno deste tópico, acidentes, neste caso em específico, relacionados com aeronaves, sejam evitados com maior eficiência. Uma abordagem para lidar com o crescimento de dados poderia ser a utilização de role-playing para as dimensões, permitindo assim poupar recursos, pois os dados ocupariam menos espaço e as interrogações teriam um melhor desempenho.

<h2> 3. Do a critical assessment of this full project </h2>

* <h3>FASE 1</h3>
O star schema estava pobre, o relatório poderia estar mais elaborado e a pesquisa que efetuámos não foi completa deixando algumas questões em aberto, como por exemplo o uso de Datasets acessórios.
<br><br>
<b>Auto-avaliação:</b> 13
<br>
Nesta fase ainda não tinhamos compreendido bem o projeto em sí resultando numa fase mais fraca.



* <h3>FASE 2</h3>
Melhorámos o star schema, adicionámos e modificámos atributos e medidas. Achamos que efetuámos de forma correta o script desenvolvido para criação de tabelas e inserção de dados. Ao querermos adicionar mais atributos à dimensão Location para conseguirmos obter mais dados optámos por recorrer à biblioteca geopy que limitou a inserção de dados (demora tempo a extrair informação).  
<br><br>
<b>Auto-avaliação:</b> 16
<br>
Demonstrámos melhorias significativas em relação à primeira fase e conseguimos implementar o que era pedido para esta fase.

* <h3>FASE 3</h3>
Um dos problemas que encontrámos, como referimos na fase 2 (o problema do geopy), prende-se com o facto de termos poucos dados inseridos na base de dados, não conseguindo ter uma melhor noção dos resultados obtidos e possíveis bottlenecks. Alterámos algumas queries pois verificámos que se poderiam adequar mais aos dados obtidos e acrescentámos outras que achámos relevantes. Foram executadas as análises de performance tal como pedido, chegando à conclusão que devido à escassez de dados inseridos, os resultados não evidenciavam necessidade de melhoria das queries, como por exemplo, através do uso de índices. 
<br><br>
<b>Auto-avaliação:</b> 14
<br>
Cumprimos com o que era pedido nesta fase, apesar das adversidades acima referidas.
